<a href="https://colab.research.google.com/github/chunseoklee/CodeMirror/blob/master/qwen3vl_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "Qwen/Qwen3-VL-2B-Instruct"
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

print("Model loaded successfully!")
print(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

Some weights of Qwen3VLModel were not initialized from the model checkpoint at Qwen/Qwen3-VL-2B-Instruct and are newly initialized: ['language_model.embed_tokens.weight', 'language_model.layers.0.input_layernorm.weight', 'language_model.layers.0.mlp.down_proj.weight', 'language_model.layers.0.mlp.gate_proj.weight', 'language_model.layers.0.mlp.up_proj.weight', 'language_model.layers.0.post_attention_layernorm.weight', 'language_model.layers.0.self_attn.k_norm.weight', 'language_model.layers.0.self_attn.k_proj.weight', 'language_model.layers.0.self_attn.o_proj.weight', 'language_model.layers.0.self_attn.q_norm.weight', 'language_model.layers.0.self_attn.q_proj.weight', 'language_model.layers.0.self_attn.v_proj.weight', 'language_model.layers.1.input_layernorm.weight', 'language_model.layers.1.mlp.down_proj.weight', 'language_model.layers.1.mlp.gate_proj.weight', 'language_model.layers.1.mlp.up_proj.weight', 'language_model.layers.1.post_attention_layernorm.weight', 'language_model.layer

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Model loaded successfully!
Qwen3VLModel(
  (visual): Qwen3VLVisionModel(
    (patch_embed): Qwen3VLVisionPatchEmbed(
      (proj): Conv3d(3, 1024, kernel_size=(2, 16, 16), stride=(2, 16, 16))
    )
    (pos_embed): Embedding(2304, 1024)
    (rotary_pos_emb): Qwen3VLVisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-23): 24 x Qwen3VLVisionBlock(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): Qwen3VLVisionAttention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (mlp): Qwen3VLVisionMLP(
          (linear_fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (linear_fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (act_fn): GELUTanh()
        )
      )
    )
    (merger): Qwen3VLVisionPatchMerger(
      (norm): LayerN

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load the model and tokenizer (if not already loaded in the current session)
model_name = "Qwen/Qwen3-VL-2B-Instruct"
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Extract the deepstack merger list
deepstack_merger = model.visual.deepstack_merger_list

# Determine the expected input shape for Qwen3VLVisionPatchMerger
# Based on the model architecture, the input feature dimension for the merger is 4096.
# Let's assume a batch size of 1 and a sequence length (number of patches) of 256 for a dummy input.
batch_size = 1
sequence_length = 256 # Example sequence length for visual features
hidden_size = 4096 # From the LayerNorm((4096,)) and linear_fc1 in_features=4096

dummy_input = torch.randn(batch_size, sequence_length, hidden_size)

print(f"Dummy input shape for deepstack_merger: {dummy_input.shape}")

# Pass the dummy input through the deepstack_merger_list
# The deepstack_merger_list contains 3 Qwen3VLVisionPatchMerger modules
# Each merger takes the input and processes it, likely passing it to the next
# For simplicity, we'll process it through the first module as an example
# If all modules process sequentially on the same input, we'd loop.
# However, typically, a ModuleList is iterated over.

output_from_merger = dummy_input
for i, merger_module in enumerate(deepstack_merger):
    print(f"Processing with deepstack_merger_list[{i}]")
    output_from_merger = merger_module(output_from_merger)
    print(f"Output shape after deepstack_merger_list[{i}]: {output_from_merger.shape}")

print(f"Final output shape from deepstack_merger_list: {output_from_merger.shape}")


In [ ]:
import safetensors.torch

# Extract the vision encoder
vision_encoder = model.visual

# Define a path to save the vision encoder in safetensors format
save_path = "./qwen3_vl_vision_encoder.safetensors"

# Save the state dictionary of the vision encoder using safetensors
safetensors.torch.save_file(vision_encoder.state_dict(), save_path)

print(f"Vision encoder saved successfully to {save_path} in safetensors format!")
print("To load the vision encoder later, you can use:")
print("new_vision_encoder = Qwen3VLVisionModel(...)")
print("new_vision_encoder.load_state_dict(safetensors.torch.load_file(\"{save_path}\"))")
print("new_vision_encoder.eval()")

Vision encoder saved successfully to ./qwen3_vl_vision_encoder.safetensors in safetensors format!
To load the vision encoder later, you can use:
new_vision_encoder = Qwen3VLVisionModel(...)
new_vision_encoder.load_state_dict(safetensors.torch.load_file("{save_path}"))
new_vision_encoder.eval()


In [ ]:
import safetensors.torch

# Extract the deepstack merger list
deepstack_merger = model.visual.deepstack_merger_list

# Define a path to save the deepstack merger in safetensors format
save_path_deepstack = "./qwen3_vl_deepstack_merger.safetensors"

# Save the state dictionary of the deepstack merger using safetensors
safetensors.torch.save_file(deepstack_merger.state_dict(), save_path_deepstack)

print(f"Deepstack merger saved successfully to {save_path_deepstack} in safetensors format!")
print("To load the deepstack merger later, you can use:")
print("from transformers.models.qwen3_vl.modeling_qwen3_vl import Qwen3VLVisionPatchMerger")
print("new_deepstack_merger = torch.nn.ModuleList([Qwen3VLVisionPatchMerger(...) for _ in range(3)])") # Adjust based on actual structure if needed
print("new_deepstack_merger.load_state_dict(safetensors.torch.load_file(\"{save_path_deepstack}\"))")
print("new_deepstack_merger.eval()")

Deepstack merger saved successfully to ./qwen3_vl_deepstack_merger.safetensors in safetensors format!
To load the deepstack merger later, you can use:
from transformers.models.qwen3_vl.modeling_qwen3_vl import Qwen3VLVisionPatchMerger
new_deepstack_merger = torch.nn.ModuleList([Qwen3VLVisionPatchMerger(...) for _ in range(3)])
new_deepstack_merger.load_state_dict(safetensors.torch.load_file("{save_path_deepstack}"))
new_deepstack_merger.eval()


In [ ]:
from transformers.models.qwen3_vl.modeling_qwen3_vl import Qwen3VLVisionPatchMerger
from transformers import Qwen3VLConfig # Qwen3VLConfig는 모델의 전체 설정 파일에서 가져옵니다.
import torch
import safetensors.torch

# model.config에서 vision_config를 가져옵니다.
vision_config = model.config.vision_config

# deepstack_merger_list를 위한 config를 생성합니다.
# Qwen3VLVisionPatchMerger의 norm layer의 hidden_size는 4096이어야 합니다.
# 따라서 vision_config를 복사하고 hidden_size를 4096으로 설정합니다.
deepstack_merger_config = Qwen3VLConfig(**vision_config.to_dict()) # Create a copy
deepstack_merger_config.hidden_size = 4096 # Set the correct hidden_size for the deepstack merger

# Qwen3VLVisionPatchMerger 인스턴스를 올바른 config 객체로 초기화합니다.
# deepstack_merger_list에는 3개의 Qwen3VLVisionPatchMerger가 있습니다.
new_deepstack_merger = torch.nn.ModuleList([
    Qwen3VLVisionPatchMerger(deepstack_merger_config) for _ in range(3)
])

# 저장된 state_dict를 로드합니다.
new_deepstack_merger.load_state_dict(safetensors.torch.load_file("./qwen3_vl_deepstack_merger.safetensors"))
new_deepstack_merger.eval()

print("Deepstack merger loaded and configured successfully!")

NameError: name 'model' is not defined